This notebook follows 05_make_rentina_subsets_for_teaching.

Inputs:
1. retina_clusteridentities.txt
2. amacrine_gene_metadata.csv
3. differential_clusters_pca_lowrank.csv

This notebook outputs the following files:
1. differential_clusters_lowrank_tidy.csv
2. amacrine_lowrank_tidy.csv

Comment: I have no idea why we have to save these. They files are huge.

In [1]:
import pandas as pd
import numpy as np

In [2]:
to_save=False  # do not save this, it's seemingly pointless

# Get Data

In [11]:
# cell_cluster information
cluster_df = pd.read_csv("data/downloads/retina_clusteridentities.txt",
                         sep='\t', header=None, names=['cell', 'cluster_no'], index_col=0)

# amacrine_gene_metadata
amacrine_gene_metadata = pd.read_csv('data/amacrine/amacrine_gene_metadata.csv', index_col=0)

# Lowrank
lowrank_df = pd.read_csv('data/pca/differential_clusters_pca_lowrank.csv', index_col=0)

# Tidify and Log Expression

In [6]:
# tidify and log
lowrank_tidy = lowrank_df.unstack().reset_index()
lowrank_tidy = lowrank_tidy.rename(columns={'level_0': 'gene_symbol',
                                            'cell': 'barcode',
                                            0: 'expression'})
lowrank_tidy['expression_log'] = np.log10(lowrank_tidy['expression'] + 1)  # in the original notebook this was exported too

lowrank_tidy = lowrank_tidy.join(cluster_df, on='barcode')  # add cluster_no

# preview
print(lowrank_tidy.shape)
lowrank_tidy.head()

/home/harrisonized/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


(7645400, 5)


,gene_symbol,barcode,expression,expression_log,cluster_no
0,1500015O10RIK,r1_GGCCGCAGTCCG,0.0,0.0,2
1,1500015O10RIK,r1_CTTGTGCGGGAA,0.0,0.0,2
2,1500015O10RIK,r1_GCGCAACTGCTC,0.0,0.0,2
3,1500015O10RIK,r1_GATTGGGAGGCA,-0.0,0.0,2
4,1500015O10RIK,r1_GTGCCGCCTCTC,-0.0,0.0,25


In [7]:
if to_save:
    lowrank_tidy_metadata.to_csv('data/pca/differential_clusters_pca_lowrank_tidy.csv')

# Amacrine

In [9]:
# subset
amacrine_cluster_nos = list(range(3, 24))
amacrine_lowrank_tidy = lowrank_tidy[
    (lowrank_tidy['cluster_no'].isin(amacrine_cluster_nos)) &  # select amacrine clusters
    (lowrank_tidy['gene_symbol'].isin(amacrine_gene_metadata.index))  # select amacrine genes
]

# preview
print(amacrine_lowrank_tidy.shape)
amacrine_lowrank_tidy.head()

(438396, 5)


,gene_symbol,barcode,expression,expression_log,cluster_no
12055,1700025G04RIK,r1_GGGTGTCAGTGG,-0.045608,-0.020273,6
12058,1700025G04RIK,r1_GTTTATATGCGC,0.178449,0.071311,5
12059,1700025G04RIK,r1_TCTTCACTGGCT,0.252604,0.097814,13
12060,1700025G04RIK,r1_TCATTTAGTCGA,0.000000,0.000000,8
12063,1700025G04RIK,r1_GTCTATTCGGTT,-0.132676,-0.061818,10


In [10]:
if to_save:
    lowrank_tidy_amacrine.to_csv('data/pca/amacrine_pca_lowrank_tidy.csv')